In [ ]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

import matplotlib.pyplot as plt

In [ ]:
submission_files = [
    "/opt/ml/level2_objectdetection-cv-01/work_dirs/swin-t-p4-w7_cascade_rcnn_fpn_adamW_focalLoss_CosRes_mstrain_augmentation/submission_latest.csv",
    "/opt/ml/level2_objectdetection-cv-01/work_dirs/swin-t-p4-w7_cascade_rcnn_Non_Local_fpn_adamW_focalLoss_CosRes_mstrain/submission_latest.csv"
    ]
submission_df = [pd.read_csv(file) for file in submission_files]

In [ ]:
submission_df

In [ ]:
annotation = '../dataset/test.json'
coco = COCO(annotation)
image_ids = submission_df[0]['image_id'].tolist()

In [ ]:
prediction_strings = []
file_names = []
iou_thr = 0.65
skip_box_thr = 0.0001
weights = [4, 2]

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
        
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr) 
        # boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [ ]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('./Ensemble/submission_ensemble.csv')

submission.head()
len(submission)

In [ ]:
category = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

fig, axes = plt.subplots(1, 2, figsize=(25, 10))

confidences = [{i:[] for i in range(10)}, {i:[] for i in range(10)}]

for n, df in enumerate(submission_df):
    for lst in df["PredictionString"].tolist():
        predict = str(lst).split()
        for i in range(len(predict)//6):
            label, confidence = int(predict[6*i + 0]), float(predict[6*i + 1])
            confidences[n][label].append(confidence)

tmp1 = [v if v else 0 for _, v in confidences[0].items()]
tmp2 = [v if v else 0 for _, v in confidences[1].items()]

axes[0].boxplot(tmp1)
axes[1].boxplot(tmp2)
axes[0].set_xticks(range(1, len(category)+1))
axes[1].set_xticks(range(1, len(category)+1))
axes[0].set_xticklabels(category)
axes[1].set_xticklabels(category)
plt.show()


In [ ]:
import os

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

In [ ]:
colors = ['cyan', 'lime', 'orange', 'deeppink', 'gold', 'blueviolet', 'red', 'teal', 'hotpink', 'mediumspringgreen']

fig, axes = plt.subplots(4, 3, figsize=(15, 20))

informations = {f"train{idx}" : df for idx, df in enumerate(submission_df)}
informations['ensemble'] = submission
for y, image_id in enumerate(image_ids[16:20]):
    img_path = os.path.join("../dataset/", image_id)
                            
    for i, (name, df) in enumerate(informations.items()):
        predict_list = str(df[df['image_id'] == image_id]['PredictionString'].tolist()[0]).split()
        x = i % 3
        img = Image.open(img_path)

        axes[y][x].imshow(img)
        axes[y][x].axis('off')
        axes[y][x].set_title(name)

        for idx in range(len(predict_list)//6):
            label, confidence, x_min, y_min, x_max, y_max = int(predict_list[6*idx + 0]), float(predict_list[6*idx + 1]), float(predict_list[6*idx + 2]), float(predict_list[6*idx + 3]), float(predict_list[6*idx + 4]), float(predict_list[6*idx + 5])
            rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=1, edgecolor=colors[label], facecolor='none')
            axes[y][x].add_patch(rect)
            axes[y][x].text(x_min, y_min-10, f"{category[label]}", color=colors[label], fontsize=7, weight='bold')
fig.set_tight_layout
plt.show()